# This notebook shows how to run flant5-xxl on the prompts and get the needed outputs

In [12]:
from stance_detector.flant5_runner import FlanT5Runner
import yaml
import pprint as p

In [2]:
config_path = "../config.yaml"
with open(config_path, "r") as f:
    config = yaml.safe_load(f)
max_memory_mapping = config["max_memory_mapping"]
print(max_memory_mapping)

{0: '0GB', 1: '0GB', 2: '0GB', 3: '0GB', 4: '0GB', 5: '0GB', 6: '0GB', 7: '80GB'}


In [ ]:
# load flant5
model_name = "google/flan-t5-xxl"
tokenizer_cache = "/home/racball/flan-t5-xxl--tokeniser"
model_cache = "/home/racball/models--flan-t5-xxl"
device_map = "sequential"
cache_dir= "/home/racball/flan-t5-xxl--tokeniser"

runner = FlanT5Runner(
    model_name=model_name,
    tokenizer_cache=tokenizer_cache,
    model_cache=model_cache,
    device_map=device_map,
    max_memory=max_memory_mapping,
    )

In [4]:
results_cf = runner.run_inference(input_path="../data/semeval_taskA_targetAtheism_prompt1_instruction1_prompts_free.parquet")

2025-05-14 14:35:42,716 - stance_detector.flant5_runner - INFO - Loaded 1 records from ../data/semeval_taskA_targetAtheism_prompt1_instruction1_prompts_free.parquet


  0%|          | 0/1 [00:00<?, ?it/s]/nobackup/racball/github/stance_detector/.venv/lib/python3.7/site-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
100%|██████████| 1/1 [00:04<00:00,  4.62s/it]

2025-05-14 14:35:47,346 - stance_detector.flant5_runner - INFO - Saved results to ../data/semeval_taskA_targetAtheism_prompt1_instruction1_prompts_free--output.pkl
2025-05-14 14:35:47,347 - stance_detector.flant5_runner - INFO - Processed 1 records


In [5]:
results_cf

,ID,Target,Tweet,Stance,Prompt,label,class_tokens,generated_token,generated_out_tokens_log_prob,generated_overall_log_prob,class_token_log_probs,class_token_log_probs_total
0,10001,Atheism,He who exalts himself shall be humbled; a...,AGAINST,Your response to the question should be either...,1_free--1,"[Positive, Negative, Neutral, Positive., Negat...",neutral,"{'neutral': -0.7013075351715088, '</s>': -2.12...",-0.701329,"[{'Positive': -6.235310077667236, '</s>': -2.1...","[-6.235331297146331, nan, nan, nan, nan, nan, ..."


- `ID` is the datapoint ID

- `Target` is the target towards which stance is being inferred

- `Tweet` is the tweet to be used to infer stance.

- `label` is of the type x--y where x is an indentifier for the prompt_template and y is the identifier for instruction.

- `Stance` is the ground truth stance label

- `Prompt` is the prompt to the LLM

- `class_tokens` refers to the set of candidate tokens (options) for which we compute the probabilities that they are the outputs of the LLM given the prompt, i.e., P(y|x).

- **`generated token` is the output by greedy decoding**

- `generated_out_tokens_log_prob` is the log probability of the tokens making up the output by greedy decoding

- `generated_overall_log_prob` is the total log probability obtained by summing up the tokens making up the output by greedy decoding

- `class_token_log_probs` contains the log probabilities of the tokens making up options of interest to us.

- `class_token_log_probs_total` is the total log probabilities obtained by summing up the tokens making up the options of interest to us.

In [ ]:
print(results_cf.Prompt.iloc[0])

Your response to the question should be either positive, negative, or neutral.

Statement: 
Question: What is the attitude of the statement to Atheism?


In [ ]:
# this is the token which flant5-xxl outputs (in a greedy setting)
print(results_cf.generated_token.iloc[0])

neutral


In [ ]:
# this is the log probabilities of the answer (generated_token) to the prompt
print(results_cf.generated_out_tokens_log_prob.iloc[0])

{'neutral': -0.7013075351715088, '</s>': -2.1219479094725102e-05}


In [ ]:
# this is the sum of the log probabilities of the generated_tokens to the prompt
print(results_cf.generated_overall_log_prob.iloc[0])

-0.7013287546506035


In [ ]:
# this is the log probabilities of tokens which make up the options in the prompt
p.pprint(results_cf.class_token_log_probs.iloc[0])

[{'</s>': -2.1219479094725102e-05, 'Positive': -6.235310077667236},
 {'</s>': nan, 'Neg': -6.172260284423828, 'ative': -19.49579429626465},
 {'</s>': nan, 'Neu': -5.477845191955566, 'tral': -24.205183029174805},
 {'.': -12.03813648223877, '</s>': nan, 'Positive': -6.235310077667236},
 {'.': nan,
  '</s>': nan,
  'Neg': -6.172260284423828,
  'ative': -19.49579429626465},
 {'.': nan,
  '</s>': nan,
  'Neu': -5.477845191955566,
  'tral': -24.205183029174805},
 {'</s>': -2.1219479094725102e-05, 'positive': -1.5652083158493042},
 {'</s>': -2.1219479094725102e-05, 'negative': -1.3100529909133911},
 {'</s>': -2.1219479094725102e-05, 'neutral': -0.7013075351715088},
 {'.': -12.03813648223877, '</s>': nan, 'positive': -1.5652083158493042},
 {'.': -12.03813648223877, '</s>': nan, 'negative': -1.3100529909133911},
 {'.': -12.03813648223877, '</s>': nan, 'neutral': -0.7013075351715088}]


In [ ]:
# this is the sum of the log probabilities of tokens which make up the options in the prompt
p.pprint(results_cf.class_token_log_probs_total.iloc[0])

[-6.235331297146331,
 nan,
 nan,
 nan,
 nan,
 nan,
 -1.565229535328399,
 -1.3100742103924858,
 -0.7013287546506035,
 nan,
 nan,
 nan]
